## Download required data from yahoo finance

### Import libraries

In [1]:
import yfinance as yf
import pandas as pd

### Download the stock data

In [2]:
# tickers is the list of stock tickers of 15 companies listed on the NYSE
tickers = ["AAPL", "AMZN", "META", "MSFT", "GOOG", "NVDA", "NFLX", "DIS", "TSLA", "LLY", "JNJ", "BA", "JPM", "V", "MA"]
start_date = "2018-01-01"
end_date = "2022-12-31"
for ticker in tickers:
    # download the data from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date) 
    # convert the downloaded data into csv files
    data.to_csv("stock_data/{}_{}-{}".format(ticker, start_date[:4], end_date[:4]))
print("Data added to stock_data folder")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Data added to stock_data folder


## twitter data

In [2]:
import tweepy

consumer_key = "tmYCkfjMIpan4sz6tRktSxAFj"
consumer_secret = "LPN1LFuq4sankbmubMRSABV1w2o4TDhbIrjsAemdu8EEpuNXEo"
access_token = "1522464891931570176-gKqi24Xqd4b8LxXOEmB94QlDaz2igx"
access_token_secret = "ze846fgmo9tLiaLUoZfKGWKDzxR9JGIc56FXQyTPSp2FW"

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify = True)
api = tweepy.API(auth=auth, wait_on_rate_limit=True)

## code from tweepy documentation
# import tabulate
# user = api.get_user()
# print("User Details: ")
# print(tabulate([["Name","Description","Location"],[user.name,user.description,user.location]],headers="firstrow"))
# Search for tweets


query = 'python programming'  # Your search query
max_results = 10  # Maximum number of results
tweets = tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended').items(max_results)

# Save tweets to a text file
with open('tweets.txt', 'w', encoding='utf-8') as file:
    for tweet in tweets:
        file.write(tweet.full_text + '\n')

print(f'{max_results} tweets saved to tweets.txt')

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product